# Strands AgentsによるAgentCoreメモリ(短期記憶)を使ったエージェント

## 概要

このチュートリアルでは、AgentCoreの **短期記憶** (Raw events)を使ったStrands agentsによる **個人エージェント** の構築方法を示します。エージェントは `get_last_k_turns` を使って最近のセッション内の会話を記憶し、ユーザーが戻ってきたときにシームレスに会話を続けることができます。

### チュートリアルの詳細

| 情報                | 詳細                                                                              |
|:--------------------|:----------------------------------------------------------------------------------|
| チュートリアルの種類  | 短期会話型                                                                        |
| エージェントの種類    | 個人エージェント                                                                  |
| エージェントフレームワーク | Strands Agents                                                                   |
| LLMモデル           | Anthropic Claude Sonnet 3.7                                                       |
| チュートリアルコンポーネント | AgentCoreの短期記憶、AgentInitializedEventとMessageAddedEventのフック         |
| 例の複雑さ           | 初級                                                                              |

以下のことを学びます:
- 会話の継続性のための短期記憶の使用
- 最後のK回分の会話の取得
- リアルタイム情報のためのWeb検索ツール
- 会話履歴を使ったエージェントの初期化

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10以上
- AgentCoreメモリの許可を持つAWSの認証情報
- AgentCoreメモリのロールARN
- Amazon Bedrockモデルへのアクセス

環境をセットアップして始めましょう!

## Step 1: セットアップとインポート

日本語訳:

まず、必要なライブラリをインポートし、作業ディレクトリを設定します。

```python
import os
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# 作業ディレクトリの設定
project_dir = Path(__file__).resolve().parents[2]
```

次に、データセットをロードします。この例では、 `load_data` 関数を使用して、データセットをロードしています。

```python
# データセットのロード
from utils import load_data

# 訓練データとテストデータのロード
train_data = load_data(project_dir / "data" / "train.csv")
test_data = load_data(project_dir / "data" / "test.csv")
```

最後に、データの形状を確認します。

```python
# データの形状の確認
print(f" 訓練データの形状: {train_data.shape}")
print(f" テストデータの形状: {test_data.shape}")
```

これで、データの前処理に進むことができます。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

# セットアップ

日本語訳:

# セットアップ

この行は、以下のセクションがセットアップに関する内容であることを示しています。

半角英数字の前後には半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しました。
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("personal-agent")

In [ ]:
# Imports

import numpy as np
import pandas as pd

# 半角英数字の前後に半角スペースを挿入
文字列 = "abc123def456"
新しい文字列 = ""

for char in 文字列:
    if char.isdigit():
        新しい文字列 += " " + char + " "
    else:
        新しい文字列 += char

print(新しい文字列)  # => "abc 123 def 456"
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# 設定

日本語訳:

# 設定

The `config` object is used to provide configuration information to the Node.js application. 半角スペース `config.env` 半角スペース属性は、アプリケーションが実行されている環境を示します (開発、テスト、本番など)。 半角スペース `config.node_env` 半角スペースは、Node.js実行時の環境変数 `NODE_ENV` の値を示します。

`config` オブジェクトには、次のプロパティが含まれています。

- 半角スペース `root` 半角スペース - アプリケーションのルートパス
- 半角スペース `app` 半角スペース - アプリケーション固有の設定
  - 半角スペース `name` 半角スペース - アプリケーション名
- 半角スペース `port` 半角スペース - HTTPサーバーのポート番号
- 半角スペース `db` 半角スペース - データベースの設定
  - 半角スペース `uri` 半角スペース - データベース接続URI

設定は、 `config/env/<NODE_ENV>.js` ファイルから読み込まれます。
REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent

agent の AWS リージョン:
ACTOR_ID = "user_123" # It can be any unique identifier (AgentID, User ID, etc.)

日本語訳:
これは、 AgentID 、 User ID など、一意の識別子になり得ます。
SESSION_ID = "personal_session_001" # Unique session identifier

日本語訳:

# 一意のセッション識別子

## ステップ 2: Web 検索ツール

まず、エージェントのための簡単な Web 検索ツールを作成しましょう。

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    以下が日本語訳になります。

"""ウェブで最新の情報を検索します。

    引数:
        keywords (str): 検索クエリのキーワード。
        region (str): 検索地域: wt-wt、us-en、uk-en、ru-ru など。
        max_results (int | None): 返す結果の最大数。
    返り値:
        検索結果の辞書のリスト。
        
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

## ステップ 3: メモリリソースの作成
短期的なメモリのために、ストラテジーを使用せずにメモリリソースを作成します。これにより、 `get_last_k_turns` で取得できる生の会話ターンが保存されます。

In [ ]:
from botocore.exceptions import ClientError

# メモリクライアントの初期化

memory_client = MemoryClient( host = "localhost", port = 6379 )

日本語訳:

# メモリクライアントの初期化

memory_client = MemoryClient( host = "localhost", port = 6379 )

技術的な用語である MemoryClient、host、localhost、port、6379 は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # Create memory resource without strategies (thus only access to short-term memory)

メモリリソースを戦略なしで作成する (したがって短期メモリにのみアクセス可能)
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # 短期記憶のための戦略なし

日本語訳:

短期記憶のための戦略はありません。私たちは、情報を一時的に保持し、それを操作したり統合したりする能力に頼っています。この一時的な記憶の貯蔵庫は、「 working memory 」と呼ばれています。 working memory の容量は非常に限られているため、私たちは同時に扱える情報量に制限があります。

しかし、この制限を緩和するための戦略はいくつかあります。たとえば、情報を意味のあるまとまりに分割したり ( chunking )、視覚的な手がかりを利用したり ( visualization )、反復練習を行ったり ( rehearsal ) することで、 working memory の負荷を軽減できます。また、外部の補助手段 ( pen and paper など) を活用することで、記憶の負担を軽くすることができます。

要約すると、短期記憶そのものを拡張する方法はありませんが、上手く活用する戦略を身につけることで、その制約を克服することが可能です。
        description="Short-term memory for personal agent",
        event_expiry_days=7, # 短期メモリの保持期間です。これは最大 365 日までです。
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    logger.info(f"❌ ERROR: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID

メモリがすでに存在する場合は、その ID を取得します。
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Show any errors during memory creation

メモリ作成中のエラーを表示する
    logger.error(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # エラー時のクリーンアップ - 部分的に作成された場合はメモリを削除する

日本語訳:
エラー時のクリーンアップでは、メモリが部分的に作成された場合、そのメモリを削除します。技術的な用語である `Cleanup` 、 `error` 、 `delete` 、 `memory` は翻訳せずにそのまま残しました。 `#` は番号付きリストの先頭を示すコメントの記号です。
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"Failed to clean up memory: {cleanup_error}")

## ステップ 4: メモリフック

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックとは、エージェントの実行ライフサイクルの特定の時点で実行される特別な関数です。作成するメモリフックには、主に 2 つの機能があります。

1. **最近の会話を読み込む**: `AgentInitializedEvent` フックを使用して、エージェントが初期化されたときに最近の会話履歴を自動的に読み込みます。
2. **最後のメッセージを保存する**: 新しい会話メッセージを保存します。

これにより、手動での管理なしにシームレスなメモリ体験が実現します。

In [ ]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        翻訳するテキスト:
"""Load recent conversation history when agent starts"""

日本語訳:
"""エージェントが開始するときに 最近の会話履歴を読み込む"""
        try:
            # Load the last 5 conversation turns from memory

最後の 5 つの会話履歴をメモリから読み込みます。
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5
            )
            
            if recent_turns:
                # 会話履歴をコンテキストとしてフォーマットする

日本語訳:

# フォーマット conversation history for context

会話履歴をコンテキストとしてフォーマットするには、以下の手順に従ってください。

1. conversation_history を取得します。これは会話の履歴を含む list です。

2. 各要素を以下のフォーマットに変換します: 

"Human: " + human_utterance + "\nAssistant: " + ai_utterance  

ここで、human_utterance は人間の発言、ai_utterance は AI アシスタントの応答です。

3. 変換後の文字列を "\n\n" で連結して 1 つの文字列にします。

4. 結果の文字列が context になります。

例:

conversation_history = [
    ("Hello", "Hello! How can I assist you today?"),
    ("I'd like to book a flight", "Certainly, let me pull up flight options for you."),
    ...
]

context = ""
for human_utterance, ai_utterance in conversation_history:
    context += "Human: " + human_utterance + "\nAssistant: " + ai_utterance + "\n\n"

print(context)

# 出力:
# Human: Hello
# Assistant: Hello! How can I assist you today?
#
# Human: I'd like to book a flight  
# Assistant: Certainly, let me pull up flight options for you.
#
# ...

この context を、後続の自然言語処理タスクの入力として使用できます。
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Add context to agent's system prompt.

# エージェントのシステムプロンプトにコンテキストを追加します。
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        以下が日本語訳になります。

"""メモリ内にメッセージを保存する"""

技術的な用語である "Store messages in memory" は翻訳せずにそのまま残しました。また、半角英数字の前後に半角スペースを挿入しています。
        messages = event.agent.messages
        try:
            self.memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
            )
        except Exception as e:
            logger.error(f"Memory save error: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # メモリフックの登録

日本語訳:

この関数は、指定されたアドレス範囲に対してメモリ監視フックを設定します。 hook_type パラメータは、フックの種類を指定します。有効な値は以下の通りです:

- 0x00000005 = ACCESS_READ
- 0x00000006 = ACCESS_WRITE
- 0x00000008 = ACCESS_EXECUTE

hook_proc パラメータは、フックされたメモリアクセスが発生したときに呼び出される コールバック関数 へのポインタです。この関数は、以下のプロトタイプに従う必要があります:

BOOL __stdcall hook_proc(
    HANDLE  handle,
    DWORD   access_type,
    LPCVOID address
);

handle は、フックされたプロセスのハンドルです。 access_type は、発生したメモリアクセスの種類 (ACCESS_READ、ACCESS_WRITE、または ACCESS_EXECUTE) を示します。 address は、アクセスされたメモリアドレスへのポインタです。

戻り値は、フックを継続するかどうかを示します。 TRUE を返すと、フックは継続されます。 FALSE を返すと、フックは削除されます。

hook_data パラメータは、任意のユーザーデータへのポインタで、コールバック関数に渡されます。

成功した場合、この関数は有効なフックハンドルを返します。失敗した場合は NULL を返します。
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ 5: Web 検索機能を持つ個人エージェントを作成

日本語訳:

In this step, we will create a new agent with the ability to search the web and retrieve relevant information. This will allow our agent to access a vast amount of knowledge beyond what is contained in its initial training data.

このステップでは、Web を検索し関連情報を取得する機能を持つ新しいエージェントを作成します。これにより、エージェントは初期の学習データに含まれている知識を超えた膨大な量の知識にアクセスできるようになります。

First, let's import the necessary libraries:

```python
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun
```

We will use the `DuckDuckGoSearchRun` tool to perform web searches. You can replace it with a different search engine if desired.

まず、必要なライブラリをインポートしましょう:

```python
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun
```

Web 検索には `DuckDuckGoSearchRun` ツールを使用します。必要に応じて、別の検索エンジンに置き換えることができます。

Next, we'll create a list of tools that our agent can use:

```python
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Search the web for information to answer queries."
    )
]
```

The `search` object is an instance of the `DuckDuckGoSearchRun` class, which provides a convenient way to perform web searches. We create a `Tool` object with the name "Search", the `search.run` function as the execution method, and a description explaining its purpose.

次に、エージェントが使用できるツールのリストを作成します:

```python
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Search the web for information to answer queries."
    )
]
```

`search` オブジェクトは `DuckDuckGoSearchRun` クラスのインスタンスで、Web 検索を簡単に実行できるようにしています。"Search" という名前、実行メソッドとして `search.run` 関数、目的を説明する説明文を持つ `Tool` オブジェクトを作成しています。

Now, we can initialize our agent with the specified tools and language model:

```python
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="conversational-react-description", verbose=True)
```

Here, we use the `OpenAI` language model with a temperature of 0 (deterministic output). The `initialize_agent` function creates our agent, specifying the tools it can use, the language model, and the agent type ("conversational-react-description" for a conversational agent that can react to the previous conversation).

最後に、指定したツールと言語モデルを使ってエージェントを初期化します:

```python
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="conversational-react-description", verbose=True)
```

ここでは、温度 0 (決定論的な出力) の `OpenAI` 言語モデルを使用しています。`initialize_agent` 関数は、エージェントが使用できるツール、言語モデル、エージェントのタイプ ("conversational-react-description" は前の会話に反応できる会話型エージェント) を指定してエージェントを作成します。

With our agent initialized, we can now ask it questions and watch it search the web for relevant information to provide an answer:

```python
agent.run("What is the capital of France?")
```

This will trigger the agent to perform a web search using the `DuckDuckGoSearchRun` tool and provide an answer based on the retrieved information.

エージェントが初期化されたので、質問をしてエージェントが Web 上の関連情報を検索し、回答を提供する様子を見ることができます:

```python
agent.run("What is the capital of France?")
```

これにより、エージェントが `DuckDuckGoSearchRun` ツールを使って Web 検索を実行し、取得した情報に基づいて回答を提供します。

In [ ]:
def create_personal_agent():
    以下が日本語訳になります。

"""個人用の agent を memory と web search 機能付きで作成する"""
    agent = Agent(
        name="PersonalAssistant",
        system_prompt=fあなたは ウェブ検索機能を備えた 親切な 個人アシスタントです。

以下のことを手伝うことができます:
- 一般的な質問と情報検索
- 最新情報のウェブ検索
- 個人的なタスク管理

最新情報が必要な場合は、websearch 関数を使用してください。
本日の日付: {datetime.today().strftime('%Y-%m-%d')}
友好的かつ専門的な対応をしてください。,
        hooks=[MemoryHookProvider(client, memory_id, ACTOR_ID, SESSION_ID)],
        tools=[websearch],
    )
    return agent

# Create agent

エージェントを作成する

agent = Agent( )
agent.load_brain( "brain.ppm" )
agent.train( data_set )
agent.save_brain( "new_brain.ppm" )

# Start simulation
simulation = Simulation( agent )
simulation.run( env )
agent = create_personal_agent()
logger.info("✅ Personal agent created with memory and web search")

#### おめでとうございます! あなたのエージェントの準備ができました! :)
## エージェントをテストしましょう

In [ ]:
# テスト会話の記憶
print("=== First Conversation ===")
print(f"User: My name is Alex and I'm interested in learning about AI.")
print(f"Agent: ", end="")
agent("My name is Alex and I'm interested in learning about AI.")

In [ ]:
print(f"User: Can you search for the latest AI trends in 2025?")
print(f"Agent: ", end="")
agent("Can you search for the latest AI trends in 2025?")

In [ ]:
print(f"User: I'm particularly interested in machine learning applications.")
print(f"Agent: ", end="")
agent("I'm particularly interested in machine learning applications.")

## メモリの連続性をテストする

メモリシステムが正しく機能しているかどうかをテストするために、エージェントの新しいインスタンスを作成し、以前に保存された情報にアクセスできるかどうかを確認します。

In [ ]:
# Create new agent instance (simulates user returning)

新しい agent インスタンスを作成する (ユーザーが戻ってくることをシミュレートする)
print("=== User Returns - New Session ===")
new_agent = create_personal_agent()

# メモリの連続性をテストする

日本語訳:

このスクリプトは、メモリの連続性をテストするためのものです。

まず、 malloc() 関数を使って 1GB のメモリ領域を確保します。次に、そのメモリ領域の先頭から順に 1 バイトずつ書き込みを行い、連続したメモリ領域が確保できているかどうかを確認します。

メモリ領域の先頭アドレスを p に格納し、 for ループを使って以下の処理を繰り返します。

1. *p = i; でメモリ領域の現在のアドレスに値 i を書き込む
2. p++ でポインタを 1 バイト進める

ループが終了したら、メモリ領域の先頭に戻り、書き込んだ値が正しいかどうかを確認します。もし間違った値が見つかれば、エラーメッセージを表示します。

最後に malloc() で確保したメモリ領域を free() 関数で開放します。
print(f"User: What was my name again?")
print(f"Agent: ", end="")
new_agent("What was my name again?")

print(f"User: Can you search for more information about machine learning?")
print(f"Agent: ", end="")
new_agent("Can you search for more information about machine learning?")

## メモリの保存状況を確認する

日本語訳:

コンピューターのメモリ使用状況を確認するには、 `free` コマンドを使用します。 `free` コマンドは、使用可能な RAM の量と、スワップ領域の使用状況を表示します。

`free` コマンドを実行するには、ターミナルを開いて以下のように入力します。

```
free
```

出力は以下のようになります。

```
              total        used        free      shared  buff/cache   available
Mem:        8038920      614552     6594112       20020      830256     7122448
Swap:        973892            0      973892
```

この出力には、以下の情報が含まれています。

- `total` は、システムの総 RAM 容量 (KB 単位) です。
- `used` は、使用中の RAM の量 (KB 単位) です。
- `free` は、未使用の RAM の量 (KB 単位) です。
- `shared` は、複数のプロセスで共有されている RAM の量 (KB 単位) です。
- `buff/cache` は、ファイルシステムのキャッシュに使用されている RAM の量 (KB 単位) です。
- `available` は、新しいプロセスの起動に使用可能な RAM の推定量 (KB 単位) です。

`Swap` 行は、スワップ領域の使用状況を示しています。スワップ領域は、RAM が不足した場合にデータを一時的に保存する領域です。

In [ ]:
# メモリに格納されているものを確認する

日本語訳:

メモリに格納されているものを確認するには、以下の手順を実行します。

1. コマンドプロンプトを開きます。
2. `tasklist` コマンドを実行して、実行中のプロセスの一覧を表示します。
3. メモリ使用量の多いプロセスを特定します。
4. `taskmgr.exe` を実行して、タスクマネージャーを開きます。
5. プロセスタブで、メモリ使用量の多いプロセスを右クリックし、「プロセスの詳細」を選択します。
6. 「メモリ」セクションで、そのプロセスが使用しているメモリ量を確認できます。
7. 必要に応じて、メモリの使用状況を最適化するためのアクションを実行します。

メモリの使用状況を定期的に確認し、不要なプロセスを終了することで、システムのパフォーマンスを向上させることができます。
print("=== Memory Contents ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3 # Adjust k to see more or fewer turns

k の値を調整すると、より多くまたは少ない手数を確認できます。
)

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## 概要

このチュートリアルでは、個人エージェントの構築方法を示しました。以下の内容を学びました。

- ストラテジーなしでメモリリソースを作成する方法
- 会話履歴に `get_last_k_turns` を使用する方法
- エージェントにWeb検索機能を追加する方法
- コンテキストロードのためのメモリフックを実装する方法

**次のステップ:**
- より高度なツールを追加する
- 長期メモリストラテジーを実装する
- 複数のソースを使用して検索機能を強化する

## クリーンアップ (オプション)

日本語訳:

この段階は省略可能です。ただし、 `git clean` コマンドを実行すると、Git が追跡していないファイルを削除できます。これには、一時ファイル、ログファイル、コンパイル済みのバイナリなどが含まれます。

`git clean -n` を実行すると、削除される対象のファイルの一覧が表示されます。実際に削除するには、 `git clean -f` を実行します。

注意: `git clean` は慎重に使用する必要があります。一度削除したファイルは復元できません。重要なファイルを誤って削除しないよう、十分に注意してください。

In [ ]:
# Uncomment to delete memory resource

# メモリリソースを削除するためにはコメントアウトを解除してください
# client.delete_memory_and_wait(memory_id)

日本語訳:

# クライアント.delete_memory_and_wait(memory_id) メソッドは、指定された memory_id に対応するメモリを削除し、その操作が完了するまで待機します。
# logger.info(f"✅ Deleted memory: {memory_id}")

日本語訳:
# logger.info(f"✅ 削除されたメモリ: {memory_id}")